In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark.sql.functions import split, regexp_replace
from sparkmeasure import StageMetrics


spark = SparkSession.builder.appName('q7').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()
stagemetrics = StageMetrics(spark)

stagemetrics.begin()

df_mv = (spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load("movie.csv"))

#df_tag = spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load('/home/administrator/Downloads/movielens/tag.csv')
df_rt = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('rating.csv')



#--Q7 START--------------------------------------------------------------------------------------------------------

#ratings_1995 = df_rt.filter(df_rt["timestamp"].rlike('^1995.*$'))#.select('movieId').distinct()
#ratings_1995.show()
ratings_per_year = df_rt.groupBy('userId', 'year')

#usr_ratings_1995 = ratings_1995.groupBy('userId').count()
#usr_ratings_1995.orderBy('count', ascending = False).show()



split_col =functions.split(df_rt['timestamp'], '-')
df_rt_year = df_rt.withColumn('year', split_col.getItem(0))
#>>df_rt_year.show()
#user1 = df_rt_year.filter((df_rt_year.userId == 91) & (df_rt_year.year == 2005)).count()
#print(user1)
ratings_pupy = df_rt_year.groupBy('userId', 'year').count()#rename count to numOfRatings
#>>ratings_pupy.show()

partitioning_rtngs = Window.partitionBy("year").orderBy(col("count").desc()) #rename count to numOfRatings
partitioned_rtngs=ratings_pupy.withColumn("row",row_number().over(partitioning_rtngs))
#>>partitioned_rtngs.show()


#q7
#u.orderBy('userId').show()

top_10_users_py=partitioned_rtngs.filter(col("row") <= 10)
#>>top_10_users_py.show()#df_tag.orderBy('tag').show(5)
top_10_users_py = top_10_users_py.drop("row")

#final q7
top_10_users_py.filter(top_10_users_py.year == 1995).show()


stagemetrics.end()

#print performance metrics
stagemetrics.print_report()
spark.stop()

#--Q7 END----------------------------------------------------------------------------------------------------------

+------+----+-----+
|userId|year|count|
+------+----+-----+
|131160|1995|    3|
| 28507|1995|    1|
+------+----+-----+


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2
Aggregated Spark stage metrics:
numStages => 10
numTasks => 414
elapsedTime => 62126 (1.0 min)
stageDuration => 60143 (1.0 min)
executorRunTime => 102834 (1.7 min)
executorCpuTime => 84940 (1.4 min)
executorDeserializeTime => 2311 (2 s)
executorDeserializeCpuTime => 1590 (2 s)
resultSerializationTime => 12 (12 ms)
jvmGCTime => 1859 (2 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 102 (0.1 s)
resultSize => 788501 (770.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 69206016
recordsRead => 40000529
bytesRead => 1381493186 (1317.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 4
shuffleTotalBlocksFetched => 4
shuffleLocalBlocksFetched => 4
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 312 (312 Bytes)
shuff